In [1]:
# coding: utf-8


import sys
import torch
import torch.nn as nn
from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split
import re
from collections import Counter, OrderedDict
from torchtext.vocab import vocab
from torch.utils.data import DataLoader

# # Machine Learning with PyTorch and Scikit-Learn  
# # -- Code Examples

# ## Package version checks

# Add folder to path in order to load from the check_packages.py script:


# # Chapter 15: Modeling Sequential Data Using Recurrent Neural Networks (Part 2/3)

# **Outline**
# 
# - [Implementing RNNs for sequence modeling in PyTorch](#Implementing-RNNs-for-sequence-modeling-in-PyTorch)
#   - [Project one -- predicting the sentiment of IMDb movie reviews](#Project-one----predicting-the-sentiment-of-IMDb-movie-reviews)
#     - [Preparing the movie review data](#Preparing-the-movie-review-data)
#     - [Embedding layers for sentence encoding](#Embedding-layers-for-sentence-encoding)
#     - [Building an RNN model](#Building-an-RNN-model)
#     - [Building an RNN model for the sentiment analysis task](#Building-an-RNN-model-for-the-sentiment-analysis-task)
#       - [More on the bidirectional RNN](#More-on-the-bidirectional-RNN)

# # Implementing RNNs for sequence modeling in PyTorch
# 
# ## Project one: predicting the sentiment of IMDb movie reviews
# 
# ### Preparing the movie review data
# 
# 

# Step 1: load and create the datasets

train_dataset = IMDB(split='train')
test_dataset = IMDB(split='test')

torch.manual_seed(1)
train_dataset, valid_dataset = random_split(
    list(train_dataset), [20000, 5000])

def convert_labels(dataset):
    return [(0 if label == 2 else 1, text) for label, text in dataset]

# Apply conversion
train_dataset = convert_labels(train_dataset)
test_dataset = convert_labels(test_dataset)




## Step 2: find unique tokens (words)

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))




## Step 3: encoding each unique token into integers

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vocab = vocab(ordered_dict)

vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['this', 'is', 'an', 'example']])




## Step 3-A: define the functions for transformation

device = torch.device("cuda:0")
# device = 'cpu'

text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1. if x == 'pos' else 0.


## Step 3-B: wrap the encode and transformation function
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), 
                                      dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)




## Step 4: batching the datasets

batch_size = 32  

train_dl = DataLoader(train_dataset, batch_size=batch_size,
                      shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,
                      shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)


# ### Embedding layers for sentence encoding
# 
# 
#  * `input_dim`: number of words, i.e. maximum integer index + 1.
#  * `output_dim`: 
#  * `input_length`: the length of (padded) sequence
#     * for example, `'This is an example' -> [0, 0, 0, 0, 0, 0, 3, 1, 8, 9]`   
#     => input_lenght is 10
#  
#  
# 
#  * When calling the layer, takes integr values as input,   
#  the embedding layer convert each interger into float vector of size `[output_dim]`
#    * If input shape is `[BATCH_SIZE]`, output shape will be `[BATCH_SIZE, output_dim]`
#    * If input shape is `[BATCH_SIZE, 10]`, output shape will be `[BATCH_SIZE, 10, output_dim]`






# ### Building an RNN model
# 
# * **RNN layers:**
#   * `nn.RNN(input_size, hidden_size, num_layers=1)`
#   * `nn.LSTM(..)`
#   * `nn.GRU(..)`
#   * `nn.RNN(input_size, hidden_size, num_layers=1, bidirectional=True)`
#  
#  



## An example of building a RNN model
## with simple RNN layer

# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, 
                          hidden_size, 
                          num_layers=2, 
                          batch_first=True)
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        _, hidden = self.rnn(x)
        out = hidden[-1, :, :]
        out = self.fc(out)
        return out

model = RNN(64, 32) 

print(model) 
 
model(torch.randn(5, 3, 64)) 


# ### Building an RNN model for the sentiment analysis task



class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
         
vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)

def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for i, (text_batch, label_batch, lengths) in enumerate(dataloader):
        optimizer.zero_grad()  # Zero the gradients

        # Forward pass
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)

        # Backward pass
        loss.backward()

        # Print gradients before optimization step
        # for name, param in model.named_parameters():
        #     if param.requires_grad:
                # print(f"Before step - {name} grad: {param.grad}")

        optimizer.step()  # Update parameters

        # Print gradients after optimization step
        # for name, param in model.named_parameters():
        #     if param.requires_grad:
        #         print(f"After step - {name} grad: {param.grad}")

        # Calculate accuracy and loss
        correct_preds = ((pred >= 0.5).float() == label_batch).float().sum().item()
        total_acc += correct_preds
        total_loss += loss.item() * label_batch.size(0)
        
        # Print batch details
        # print(f"Batch {i+1}:")
        # print(f"  Predictions: {pred}")
        # print(f"  Labels: {label_batch}")
        # print(f"  Batch Accuracy: {correct_preds / label_batch.size(0):.4f}, Batch Loss: {loss.item():.4f}")
    
    avg_acc = total_acc / len(dataloader.dataset)
    avg_loss = total_loss / len(dataloader.dataset)
    print(f"Training Accuracy: {avg_acc:.4f}, Training Loss: {avg_loss:.4f}")
    return avg_acc, avg_loss

def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for i, (text_batch, label_batch, lengths) in enumerate(dataloader):
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)

            # Calculate accuracy and loss
            correct_preds = ((pred >= 0.5).float() == label_batch).float().sum().item()
            total_acc += correct_preds
            total_loss += loss.item() * label_batch.size(0)
            
            # Print batch details
            # print(f"Batch {i+1}:")
            # print(f"  Predictions: {pred}")
            # print(f"  Labels: {label_batch}")
            # print(f"  Batch Accuracy: {correct_preds / label_batch.size(0):.4f}, Batch Loss: {loss.item():.4f}")
    
    avg_acc = total_acc / len(dataloader.dataset)
    avg_loss = total_loss / len(dataloader.dataset)
    print(f"Evaluation Accuracy: {avg_acc:.4f}, Evaluation Loss: {avg_loss:.4f}")
    return avg_acc, avg_loss

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 10 

torch.manual_seed(1)
 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')
 
acc_test, _ = evaluate(test_dl)
print(f'test_accuracy: {acc_test:.4f}') 



In [ ]:
from collections import Counter

def check_data_distribution(dataloader):
    labels = []
    for _, label_batch, _ in dataloader:
        labels.extend(label_batch.cpu().numpy())
    label_counts = Counter(labels)
    print(f"Label distribution in dataset: {label_counts}")

# Assuming you have a training dataloader
check_data_distribution(train_dl)


In [ ]:
from torchtext.datasets import IMDB
from collections import Counter
import numpy as np

# Load datasets
train_dataset = list(IMDB(split='train'))
test_dataset = list(IMDB(split='test'))

# Check the distribution of labels in the training dataset
train_labels = [label for (label, _) in train_dataset]
train_label_counts = Counter(train_labels)
print(f"Training Label distribution: {train_label_counts}")

# Check the distribution of labels in the test dataset
test_labels = [label for (label, _) in test_dataset]
test_label_counts = Counter(test_labels)
print(f"Test Label distribution: {test_label_counts}")

# Check a few samples to ensure they are loaded correctly
for i in range(5):
    sample = train_dataset[i]
    print(f"Sample {i}: Text - {sample[1][:50]}, Label - {sample[0]}")
